In [1]:
from typing import Iterator, Callable
from functools import reduce

import numpy as np
import pandas as pd
import sympy as sp
import matplotlib.pyplot as plt
from scipy.optimize import least_squares

from solved_helium_eq_with_volume_effect import symargs, solved, lambdified

In [2]:
the = pd.read_excel('Data/the_helium.xlsx').set_index('photon (eV)')
the['inten_ratio'] = the['inten_2w (W/cm2)']**0.5/the['inten_w (W/cm2)']
the

,inten_w (W/cm2),inten_2w (W/cm2),coeff_s,coeff_p,coeff_d,eta_s,eta_p,eta_d,inten_ratio
photon (eV),,,,,,,,,
14.3,10000000000000,13600000000,0.003220,0.01140,-0.01090,5.36,2.26,0,1.166190e-08
15.0,10000000000000,13400000000,0.002530,0.01050,-0.01020,5.22,2.19,0,1.157584e-08
15.9,10000000000000,12500000000,0.001770,0.00944,-0.00928,5.07,2.12,0,1.118034e-08
16.0,10000000000000,12600000000,0.001690,0.00933,-0.00917,5.06,2.11,0,1.122497e-08
17.0,10000000000000,11800000000,0.000986,0.00830,-0.00824,4.94,2.06,0,1.086278e-08
18.0,10000000000000,11300000000,0.000339,0.00746,-0.00745,4.84,2.01,0,1.063015e-08
19.0,10000000000000,11100000000,-0.000392,0.00685,-0.00684,4.77,1.98,0,1.053565e-08
19.1,10000000000000,11000000000,-0.000601,0.00676,-0.00673,4.76,1.98,0,1.048809e-08
19.5,10000000000000,11400000000,-0.001020,0.00669,-0.00661,4.73,1.97,0,1.067708e-08


In [3]:
exp = pd.read_excel('Data/exp_gauss3.xlsx').set_index('dataset')
exp['photon (eV)'] = pd.Series({'good1': 15.9, 'good2': 14.3, 'good3': 19.1, 'good4': 15.9})
exp[['photon (eV)', *[k for k in exp if k.startswith('he_')]]]
exp

,he_beta1_amp_err,he_beta1_amp_value,he_beta1_freq_err,he_beta1_freq_value,he_beta1_offset_err,he_beta1_offset_value,he_beta1_shift_err,he_beta1_shift_value,he_beta1m3_amp_err,he_beta1m3_amp_value,...,ne_beta3_amp_value,ne_beta3_freq_err,ne_beta3_freq_value,ne_beta3_offset_err,ne_beta3_offset_value,ne_beta3_shift_err,ne_beta3_shift_value,ne_beta4_c_err,ne_beta4_c_value,photon (eV)
dataset,,,,,,,,,,,,,,,,,,,,,
good1,0.016970,0.534221,0,6.283185,0.012729,-0.011979,0.034757,0.236018,0.007805,0.113821,...,0.398722,0,6.283185,0.010687,-0.039238,0.037330,-0.182652,0.015320,0.432741,15.9
good2,0.005097,0.347818,0,6.283185,0.003756,-0.072793,0.015513,0.075804,0.008122,0.141339,...,0.197069,0,6.283185,0.004613,-0.012368,0.030744,-1.145020,0.003214,-0.030674,14.3
good3,0.004871,0.496982,0,6.283185,0.003367,-0.040736,0.009357,5.368111,0.003144,0.029859,...,0.267384,0,6.283185,0.002348,-0.036890,0.012146,4.939388,0.003633,0.024581,19.1
good4,0.012234,0.281430,0,6.283185,0.008562,-0.088810,0.042583,4.612789,0.003706,0.056858,...,0.233538,0,6.283185,0.006152,-0.024713,0.037645,4.182441,0.005237,0.095546,15.9


In [4]:
dataset = 'good1'
m = exp.loc[dataset]
photon = m['photon (eV)']
ref = the.loc[photon]
keys = [
#     {'d': 'beta1_amp',
#      'm': 'he_beta1_amp_value'},
    {'d': 'beta1m3_amp',
     'm': 'he_beta1m3_amp_value'},
    {'d': 'beta2',
     'm': 'he_beta2_c_value'},
    {'d': 'beta3_amp',
     'm': 'he_beta3_amp_value'},
    {'d': 'beta4',
     'm': 'he_beta4_c_value'},
]


def solveme(x):
    r, h = x
    d = lambdified(
        coeff_s=ref['coeff_s'], coeff_p=r*ref['coeff_p'], coeff_d=ref['coeff_d'],
        eta_s=ref['eta_s'], eta_p=ref['eta_p'], eta_d=ref['eta_d'], h=h,
    )
    return [
        d[k['d']] - m[k['m']]*np.sign(d[k['d']]) if k['d'].endswith('_amp') else d[k['d']] - m[k['m']]
        for k in keys
    ]


opt = least_squares(solveme, x0=[0.01, 100])
r = ref['inten_ratio']*opt.x[0]
print(opt)
print("        r, h: {:.3f}, {:.3f}".format(*opt.x))
print(" inten_ratio: {:.3e}".format(r))
print("scaled_inten: {:.3f}e+09".format((r*1e13)**2/1e9))

 active_mask: array([0., 0.])
        cost: 2.8522722877996603e-05
         fun: array([ 0.001302  , -0.00698295,  0.0001851 , -0.00256016])
        grad: array([-1.32761522e-10, -3.74683846e-12])
         jac: array([[-8.97569843e-02, -2.21114297e-03],
       [ 1.98586527e+00,  7.66107243e-03],
       [ 6.31363146e-01,  1.55534879e-02],
       [-5.41654025e+00, -2.08959312e-02]])
     message: '`gtol` termination condition is satisfied.'
        nfev: 28
        njev: 20
  optimality: 1.3276152226182614e-10
      status: 1
     success: True
           x: array([ 0.22216438, 28.79417858])
        r, h: 0.222, 28.794
 inten_ratio: 2.484e-09
scaled_inten: 0.617e+09


In [5]:
dataset = 'good2'
m = exp.loc[dataset]
photon = m['photon (eV)']
ref = the.loc[photon]
keys = [
#     {'d': 'beta1_amp',
#      'm': 'he_beta1_amp_value'},
    {'d': 'beta1m3_amp',
     'm': 'he_beta1m3_amp_value'},
    {'d': 'beta2',
     'm': 'he_beta2_c_value'},
    {'d': 'beta3_amp',
     'm': 'he_beta3_amp_value'},
    {'d': 'beta4',
     'm': 'he_beta4_c_value'},
]


def solveme(x):
    r, h = x
    d = lambdified(
        coeff_s=ref['coeff_s'], coeff_p=r*ref['coeff_p'], coeff_d=ref['coeff_d'],
        eta_s=ref['eta_s'], eta_p=ref['eta_p'], eta_d=ref['eta_d'], h=h,
    )
    return [
        d[k['d']] - m[k['m']]*np.sign(d[k['d']]) if k['d'].endswith('_amp') else d[k['d']] - m[k['m']]
        for k in keys
    ]


opt = least_squares(solveme, x0=[0.01, 100])
r = ref['inten_ratio']*opt.x[0]
print(opt)
print("        r, h: {:.3f}, {:.3f}".format(*opt.x))
print(" inten_ratio: {:.3e}".format(r))
print("scaled_inten: {:.3f}e+09".format((r*1e13)**2/1e9))

 active_mask: array([0., 0.])
        cost: 0.12508527981835832
         fun: array([-0.01429793,  0.42869996, -0.00314822,  0.25724029])
        grad: array([4.19895674e-10, 3.74920147e-13])
         jac: array([[ 8.19666684e-03, -1.05489519e-03],
       [ 5.44933780e+00,  6.15299483e-03],
       [-3.72258127e-02,  4.79088984e-03],
       [-9.08151256e+00, -1.02541818e-02]])
     message: '`gtol` termination condition is satisfied.'
        nfev: 11
        njev: 9
  optimality: 4.1989567378664105e-10
      status: 1
     success: True
           x: array([ 0.13020231, 57.65618435])
        r, h: 0.130, 57.656
 inten_ratio: 1.518e-09
scaled_inten: 0.231e+09


In [6]:
dataset = 'good3'
m = exp.loc[dataset]
photon = m['photon (eV)']
ref = the.loc[photon]
keys = [
#     {'d': 'beta1_amp',
#      'm': 'he_beta1_amp_value'},
    {'d': 'beta1m3_amp',
     'm': 'he_beta1m3_amp_value'},
    {'d': 'beta2',
     'm': 'he_beta2_c_value'},
    {'d': 'beta3_amp',
     'm': 'he_beta3_amp_value'},
    {'d': 'beta4',
     'm': 'he_beta4_c_value'},
]


def solveme(x):
    r, h = x
    d = lambdified(
        coeff_s=ref['coeff_s'], coeff_p=r*ref['coeff_p'], coeff_d=ref['coeff_d'],
        eta_s=ref['eta_s'], eta_p=ref['eta_p'], eta_d=ref['eta_d'], h=h,
    )
    return [
        d[k['d']] - m[k['m']]*np.sign(d[k['d']]) if k['d'].endswith('_amp') else d[k['d']] - m[k['m']]
        for k in keys
    ]


opt = least_squares(solveme, x0=[0.01, 100])
r = ref['inten_ratio']*opt.x[0]
print(opt)
print("        r, h: {:.3f}, {:.3f}".format(*opt.x))
print(" inten_ratio: {:.3e}".format(r))
print("scaled_inten: {:.3f}e+09".format((r*1e13)**2/1e9))

 active_mask: array([0., 0.])
        cost: 0.00018904613215515046
         fun: array([-0.01819342, -0.00659546,  0.00121098, -0.00145781])
        grad: array([-4.45811547e-11,  3.27705292e-12])
         jac: array([[ 4.96371659e-02,  7.45272053e-04],
       [ 1.33239627e+00,  3.61484090e-03],
       [ 7.45733790e-01,  1.11967428e-02],
       [-6.02803828e+00, -1.63542946e-02]])
     message: '`gtol` termination condition is satisfied.'
        nfev: 13
        njev: 11
  optimality: 4.4581154740042095e-11
      status: 1
     success: True
           x: array([ 0.202356  , 37.29325445])
        r, h: 0.202, 37.293
 inten_ratio: 2.122e-09
scaled_inten: 0.450e+09


In [7]:
dataset = 'good4'
m = exp.loc[dataset]
photon = m['photon (eV)']
ref = the.loc[photon]
keys = [
#     {'d': 'beta1_amp',
#      'm': 'he_beta1_amp_value'},
    {'d': 'beta1m3_amp',
     'm': 'he_beta1m3_amp_value'},
    {'d': 'beta2',
     'm': 'he_beta2_c_value'},
    {'d': 'beta3_amp',
     'm': 'he_beta3_amp_value'},
    {'d': 'beta4',
     'm': 'he_beta4_c_value'},
]


def solveme(x):
    r, h = x
    d = lambdified(
        coeff_s=ref['coeff_s'], coeff_p=r*ref['coeff_p'], coeff_d=ref['coeff_d'],
        eta_s=ref['eta_s'], eta_p=ref['eta_p'], eta_d=ref['eta_d'], h=h,
    )
    return [
        d[k['d']] - m[k['m']]*np.sign(d[k['d']]) if k['d'].endswith('_amp') else d[k['d']] - m[k['m']]
        for k in keys
    ]


opt = least_squares(solveme, x0=[0.01, 100])
r = ref['inten_ratio']*opt.x[0]
print(opt)
print("        r, h: {:.3f}, {:.3f}".format(*opt.x))
print(" inten_ratio: {:.3e}".format(r))
print("scaled_inten: {:.3f}e+09".format((r*1e13)**2/1e9))

 active_mask: array([0., 0.])
        cost: 0.010988373744352999
         fun: array([0.00541642, 0.1390911 , 0.00077001, 0.05099494])
        grad: array([-2.68802292e-09, -1.27401592e-11])
         jac: array([[-1.49361434e-01, -7.77433612e-04],
       [ 1.05653894e+00,  2.15982623e-03],
       [ 1.05062912e+00,  5.46857622e-03],
       [-2.88175930e+00, -5.89102652e-03]])
     message: '`gtol` termination condition is satisfied.'
        nfev: 11
        njev: 9
  optimality: 2.6880229153647406e-09
      status: 1
     success: True
           x: array([ 0.26134909, 63.92308698])
        r, h: 0.261, 63.923
 inten_ratio: 2.922e-09
scaled_inten: 0.854e+09
